# Reconocimiento de Entidades Nombradas
## Ejercicio Práctico - Procesamiento de Lenguaje Natural

**Objetivos de Aprendizaje:**
1. Implementar NER usando modelos pre-entrenados en español
2. Crear interfaces interactivas con Gradio
3. Comparar enfoques: Transformers vs API Gemini
4. Desarrollar prototipos rápidos para aplicaciones de PLN

---
**Entorno recomendado:** Google Colab o Amazon SageMaker Studio

**Tiempo estimado:** 60-90 minutos

## Instalación de Dependencias

In [1]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [2]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Todas las dependencias instaladas correctamente


## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

No se encontró GOOGLE_API_KEY
Podes continuar solo con la parte de Transformers


---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [4]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

🖥️  Dispositivo: GPU
📥 Cargando modelo de NER para español...


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Modelo mrm8488/bert-spanish-cased-finetuned-ner cargado exitosamente


In [5]:
# Texto de ejemplo con contexto argentino
texto_ejemplo = """
Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.
"""

In [6]:
def analizar_entidades_transformers(texto):
    """Procesa texto y extrae entidades usando Transformers"""
    if not ner_pipeline:
        return []

    entidades = ner_pipeline(texto)

    # Formatear resultados
    resultados = []
    for ent in entidades:
        resultados.append({
            'texto': ent['word'],
            'etiqueta': ent['entity_group'],
            'confianza': round(ent['score'], 3),
            'posicion': (ent['start'], ent['end'])
        })

    return resultados

In [7]:
# Probar el modelo
print("🔍 Analizando texto de ejemplo...")
print(f"📝 Texto: {texto_ejemplo.strip()}")
print("\n📊 Entidades encontradas:")

entidades_encontradas = analizar_entidades_transformers(texto_ejemplo)
for ent in entidades_encontradas:
    print(f"  • {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})")

🔍 Analizando texto de ejemplo...
📝 Texto: Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.

📊 Entidades encontradas:


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  • María González → PER (confianza: 0.9990000128746033)
  • Universidad de Buenos Aires → ORG (confianza: 0.9990000128746033)
  • San Telmo → LOC (confianza: 0.9980000257492065)
  • MercadoLibre → ORG (confianza: 0.996999979019165)
  • Obelisco → LOC (confianza: 0.9959999918937683)
  • Carlos Pérez → PER (confianza: 1.0)
  • Google Argentina → ORG (confianza: 0.9860000014305115)
  • Constitución → LOC (confianza: 0.9950000047683716)
  • La Boca → LOC (confianza: 1.0)


---
# PARTE 2: NER con API de Gemini

Utilizaremos la API de Gemini para un análisis más detallado y contextual.

In [8]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

API Key de Gemini no disponible
Podes obtener una gratis en: https://ai.google.dev/


In [9]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

# Probar Gemini si está disponible
if cliente_gemini:
    print("🔍 Analizando con Gemini...")
    resultado_gemini = analizar_entidades_gemini(texto_ejemplo)
    print("\nAnálisis de Gemini:")
    print(resultado_gemini)
else:
    print("⏭️  Saltando análisis con Gemini (API Key no disponible)")

⏭️  Saltando análisis con Gemini (API Key no disponible)


---
# PARTE 3: Interfaces Interactivas con Gradio

Crearemos interfaces web interactivas para probar nuestros modelos.

In [10]:
import gradio as gr

def interfaz_ner_transformers(texto):
    """Interfaz para el modelo de Transformers"""
    if not texto.strip():
        return {"text": "Ingresa un texto para analizar", "entities": []}

    if not ner_pipeline:
        return {"text": "Modelo no disponible", "entities": []}

    # Procesar con Transformers
    entidades = ner_pipeline(texto)

    # Formatear para Gradio HighlightedText
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"],
            "word": ent["word"],
            "start": ent["start"],
            "end": ent["end"],
            "score": ent["score"]
        })

    return {"text": texto, "entities": entidades_gradio}

# Ejemplos para la interfaz
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina."
]

# Crear interfaz
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers,
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText(
            label="🎯 Entidades Identificadas",
            show_legend=True
        )
    ],
    title="NER con Transformers - Español (de argentina)",
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz de Transformers creada")

✅ Interfaz de Transformers creada


In [11]:
# Interfaz para Gemini (solo si está disponible)
if cliente_gemini:
    def interfaz_ner_gemini(texto):
        """Interfaz para Gemini API"""
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto)

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox(
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

⏭️  Interfaz de Gemini no creada (API Key no disponible)


In [12]:
# Interfaz comparativa (solo si ambos están disponibles)
if ner_pipeline and cliente_gemini:
    def comparar_modelos(texto):
        """Compara resultados de ambos modelos"""
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Resultado Transformers
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Resultado Gemini
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

⏭️  Interfaz comparativa no creada (requiere ambos modelos)


## 🚀 Lanzar Interfaces

Ejecuta las celdas siguientes para lanzar las interfaces interactivas:

In [13]:
# Lanzar interfaz de Transformers
if ner_pipeline:
    print("🚀 Lanzando interfaz de Transformers...")
    demo_transformers.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: modelo de Transformers no disponible")

🚀 Lanzando interfaz de Transformers...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://016cfb8851ad7388f3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# Lanzar interfaz de Gemini
if cliente_gemini:
    print("🚀 Lanzando interfaz de Gemini...")
    demo_gemini.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: API de Gemini no disponible")

❌ No se puede lanzar: API de Gemini no disponible


In [15]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

❌ No se puede lanzar: requiere ambos modelos disponibles


---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

## 🚀 Ejercicio 3: Extensiones Avanzadas (AVANZADO)
1. Implementa procesamiento en lote de múltiples textos
2. Crea una función de exportación de resultados a CSV
3. Desarrolla un sistema de filtrado por tipo de entidad

## 💡 Proyecto Integrador
Elegí una de estas aplicaciones y desarróllala:
- **Analizador de noticias argentinas**: Extrae personas y lugares de artículos
- **Procesador de CVs**: Identifica nombres, empresas y universidades
- **Análisis de redes sociales**: Detecta menciones de políticos y lugares

## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
2. ¿En qué casos usarías un modelo local vs una API?
3. ¿Cómo evaluarías la precisión de los resultados?
4. ¿Qué consideraciones éticas debemos tener en cuenta?
5. ¿Cómo escalarías esta solución para procesar miles de documentos?

In [ ]:
# 📝 ESPACIO PARA TUS EJERCICIOS
# Usa esta celda para experimentar y desarrollar tus soluciones

# Ejemplo: Función para contar entidades por tipo
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

# Probar la función
texto_prueba = "Juan Pérez trabaja en Google Argentina en Buenos Aires con María López."
print("📊 Conteo de entidades:")
print(contar_entidades_por_tipo(texto_prueba))

# TODO: Agrega aquí tus propias funciones y experimentos

---
# 🎯 Conclusión

¡Felicitaciones! Completaste el ejercicio de Reconocimiento de Entidades Nombradas.

## 📚 Lo que aprendiste:
- ✅ Implementar NER con modelos pre-entrenados
- ✅ Usar APIs de IA generativa para tareas de PLN
- ✅ Crear interfaces interactivas con Gradio
- ✅ Comparar diferentes enfoques de NER

## 🔄 Próximos pasos:
1. Experimentá con otros modelos de Hugging Face
2. Probá con textos de diferentes dominios
3. Implementa tu proyecto integrador
4. Compartí tus resultados con la clase

## 📖 Recursos adicionales:
- [Hugging Face Models](https://huggingface.co/models?pipeline_tag=token-classification&language=es)
- [Gradio Documentation](https://gradio.app/docs/)
- [Google AI Studio](https://ai.google.dev/)

---
**¡Éxito en tu trabajo integrador!** 🎓🚀

## Ejercicio 1: Personalización (BÁSICO)

In [19]:
%%capture
from transformers import pipeline

# Crear pipeline NER (puede tardar en cargar)
ner_pipeline = pipeline("ner", grouped_entities=True, model="dslim/bert-base-NER")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [20]:
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers NER pipeline"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

In [21]:
# Lista de ejemplos con barrios de Buenos Aires
ejemplos_argentinos = [
    "El evento cultural se realizará en San Telmo este fin de semana.",
    "María vive en Palermo y trabaja en una startup en Microcentro.",
    "El colectivo 152 pasa por La Boca y luego va hacia Constitución.",
    "En Recoleta hay muchas librerías y cafés tradicionales.",
    "El concierto fue en el Teatro Gran Rex, en el centro de Buenos Aires."
]

In [22]:
# Función para detectar barrios porteños por búsqueda de palabra
barrios = ["San Telmo", "Palermo", "Microcentro", "La Boca", "Constitución", "Recoleta"]

def detectar_barrios(texto):
    encontrados = [b for b in barrios if b.lower() in texto.lower()]
    return encontrados

In [23]:
# Ejecutar conteo y detección por ejemplo
for i, texto in enumerate(ejemplos_argentinos, 1):
    conteo = contar_entidades_por_tipo(texto)
    barrios_encontrados = detectar_barrios(texto)
    print(f"\nEjemplo {i}: {texto}")
    print("Conteo de entidades:", conteo)
    print("Barrios detectados:", barrios_encontrados)


Ejemplo 1: El evento cultural se realizará en San Telmo este fin de semana.
Conteo de entidades: {'LOC': 1}
Barrios detectados: ['San Telmo']

Ejemplo 2: María vive en Palermo y trabaja en una startup en Microcentro.
Conteo de entidades: {'ORG': 2, 'LOC': 1}
Barrios detectados: ['Palermo', 'Microcentro']

Ejemplo 3: El colectivo 152 pasa por La Boca y luego va hacia Constitución.
Conteo de entidades: {'ORG': 3}
Barrios detectados: ['La Boca', 'Constitución']

Ejemplo 4: En Recoleta hay muchas librerías y cafés tradicionales.
Conteo de entidades: {'ORG': 2}
Barrios detectados: ['Recoleta']

Ejemplo 5: El concierto fue en el Teatro Gran Rex, en el centro de Buenos Aires.
Conteo de entidades: {'ORG': 2, 'LOC': 4}
Barrios detectados: []


## Ejercicio 2: Análisis Comparativo (INTERMEDIO)

In [24]:
import time

def analizar_modelos(textos, modelos):
    """
    texts: lista de textos para analizar
    modelos: dict con nombre_modelo: pipeline_ner
    """
    resultados = {}

    for nombre, modelo in modelos.items():
        print(f"\nProcesando con modelo: {nombre}")
        start_time = time.time()

        conteos = []
        for texto in textos:
            entidades = modelo(texto)
            # Dependiendo del pipeline, entidades puede tener distinto formato.
            # Normalizamos a contar el número total de entidades encontradas.
            if isinstance(entidades, list):
                if len(entidades) > 0 and isinstance(entidades[0], dict):
                    conteo_entidades = len(entidades)
                else:
                    conteo_entidades = 0
            else:
                conteo_entidades = 0
            conteos.append(conteo_entidades)

        tiempo = time.time() - start_time
        resultados[nombre] = {'conteos': conteos, 'tiempo': tiempo}
        print(f"Tiempo total: {tiempo:.3f}s")
        print(f"Entidades por texto: {conteos}")

    return resultados

In [25]:
# Ejemplo de textos para analizar
textos_ejemplo = [
    "María vive en Palermo y trabaja en Microcentro.",
    "El concierto fue en el Teatro Gran Rex en Buenos Aires.",
    "Juan Pérez trabaja en Google Argentina.",
    "El colectivo 152 pasa por La Boca.",
]

In [26]:
%%capture
# Crear pipelines (aquí tenés que ajustar a tus modelos)
from transformers import pipeline

modelo_bert = pipeline("ner", grouped_entities=True, model="dslim/bert-base-NER")
# Podés cargar otro modelo distinto, o un pipeline spaCy si querés

modelos = {
    "BERT NER": modelo_bert,
    # "Otro Modelo": otro_pipeline,  # Ejemplo para comparar
}

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [27]:
# Ejecutar análisis
resultados = analizar_modelos(textos_ejemplo, modelos)


Procesando con modelo: BERT NER
Tiempo total: 0.144s
Entidades por texto: [5, 5, 2, 2]


In [28]:
# Ejemplo simple de análisis de resultados
for modelo, data in resultados.items():
    print(f"\nModelo: {modelo}")
    print(f"Tiempo de procesamiento: {data['tiempo']:.3f} segundos")
    print(f"Entidades detectadas por texto: {data['conteos']}")


Modelo: BERT NER
Tiempo de procesamiento: 0.144 segundos
Entidades detectadas por texto: [5, 5, 2, 2]


## Ejercicio 3: Extensiones Avanzadas (AVANZADO)

In [29]:
import csv
from typing import List, Dict

In [30]:
def procesar_textos_lote(textos: List[str], modelo_ner, filtrar_tipos: List[str] = None) -> List[Dict]:
    """
    Procesa una lista de textos con un modelo NER y devuelve lista de entidades.
    Si filtrar_tipos es None, devuelve todas las entidades.
    """
    resultados = []

    for i, texto in enumerate(textos):
        entidades = modelo_ner(texto)

        for ent in entidades:
            # Dependiendo del pipeline, la estructura puede variar.
            # Aquí asumo formato dict con 'entity_group', 'start', 'end', 'word' (transformers pipeline con grouped_entities=True)
            tipo = ent.get('entity_group', '') or ent.get('entity', '')
            if filtrar_tipos is None or tipo in filtrar_tipos:
                resultados.append({
                    "texto_id": i,
                    "texto": texto,
                    "entidad": ent.get('word', '') if 'word' in ent else ent.get('text', ''),
                    "tipo": tipo,
                    "inicio": ent.get('start', -1),
                    "fin": ent.get('end', -1)
                })

    return resultados

In [31]:
def exportar_a_csv(datos: List[Dict], archivo: str):
    """
    Exporta la lista de entidades a un archivo CSV.
    """
    campos = ["texto_id", "texto", "entidad", "tipo", "inicio", "fin"]
    with open(archivo, mode='w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        for fila in datos:
            writer.writerow(fila)
    print(f"Datos exportados a {archivo}")

In [32]:
%%capture
from transformers import pipeline
modelo_ner = pipeline("ner", grouped_entities=True, model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [33]:
textos_a_procesar = [
    "El presidente Alberto Fernández visitó Buenos Aires.",
    "La Copa Libertadores fue ganada por River Plate.",
    "María compró una computadora en Mercado Libre."
]

In [34]:
# Procesar con filtro solo para entidades 'PER' (personas) y 'ORG' (organizaciones)
filtro_entidades = ['PER', 'ORG']

In [35]:
resultados_entidades = procesar_textos_lote(textos_a_procesar, modelo_ner, filtrar_tipos=filtro_entidades)

# Exportar resultados a CSV

In [36]:
# Exportar resultados a CSV
exportar_a_csv(resultados_entidades, "entidades_filtradas.csv")

Datos exportados a entidades_filtradas.csv


In [37]:
# Mostrar resultados en consola
for r in resultados_entidades:
    print(r)

{'texto_id': 0, 'texto': 'El presidente Alberto Fernández visitó Buenos Aires.', 'entidad': 'Alberto Fernández', 'tipo': 'PER', 'inicio': 14, 'fin': 31}
{'texto_id': 1, 'texto': 'La Copa Libertadores fue ganada por River Plate.', 'entidad': 'La', 'tipo': 'ORG', 'inicio': 0, 'fin': 2}
{'texto_id': 1, 'texto': 'La Copa Libertadores fue ganada por River Plate.', 'entidad': 'Libertadores', 'tipo': 'ORG', 'inicio': 8, 'fin': 20}
{'texto_id': 1, 'texto': 'La Copa Libertadores fue ganada por River Plate.', 'entidad': 'River Plate', 'tipo': 'ORG', 'inicio': 36, 'fin': 47}
{'texto_id': 2, 'texto': 'María compró una computadora en Mercado Libre.', 'entidad': 'Mercado Libre', 'tipo': 'ORG', 'inicio': 32, 'fin': 45}


## Analizador de Noticias Argentinas

In [45]:
!python -m spacy download es_core_news_sm

import spacy
import csv

# Cargar modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 79.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [46]:
def extraer_entidades_noticia(textos):
    """
    Procesa una lista de textos de noticias.
    Extrae personas (PER) y lugares (LOC).
    Devuelve lista de diccionarios con información.
    """
    resultados = []
    for i, texto in enumerate(textos):
        doc = nlp(texto)
        for ent in doc.ents:
            if ent.label_ in ["PER", "LOC", "GPE"]:  # Personas y lugares (GPE = entidades geopolíticas)
                resultados.append({
                    "id_noticia": i,
                    "entidad": ent.text,
                    "tipo": ent.label_,
                    "inicio": ent.start_char,
                    "fin": ent.end_char
                })
    return resultados

In [47]:
def exportar_resultados_csv(datos, nombre_archivo):
    campos = ["id_noticia", "entidad", "tipo", "inicio", "fin"]
    with open(nombre_archivo, mode='w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        for fila in datos:
            writer.writerow(fila)
    print(f"Archivo exportado: {nombre_archivo}")

In [51]:
# Ejemplo de uso con noticias simuladas
noticias_argentinas = [
    "El presidente Alberto Fernández visitó la provincia de Córdoba.",
    "La ciudad de Buenos Aires celebra su aniversario con múltiples eventos culturales.",
    "Lionel Messi fue homenajeado en Rosario tras ganar el premio.",
]

In [52]:
entidades_extraidas = extraer_entidades_noticia(noticias_argentinas)
exportar_resultados_csv(entidades_extraidas, "entidades_noticias.csv")

Archivo exportado: entidades_noticias.csv


In [53]:
for entidad in entidades_extraidas:
    print(entidad)

{'id_noticia': 0, 'entidad': 'Alberto Fernández', 'tipo': 'PER', 'inicio': 14, 'fin': 31}
{'id_noticia': 0, 'entidad': 'provincia de Córdoba', 'tipo': 'LOC', 'inicio': 42, 'fin': 62}
{'id_noticia': 1, 'entidad': 'ciudad de Buenos Aires', 'tipo': 'LOC', 'inicio': 3, 'fin': 25}
{'id_noticia': 2, 'entidad': 'Lionel Messi', 'tipo': 'PER', 'inicio': 0, 'fin': 12}
{'id_noticia': 2, 'entidad': 'Rosario', 'tipo': 'LOC', 'inicio': 32, 'fin': 39}
